# A/B-тест: Влияние промо-акций на продажи 🛒📈

**Дата:** 2025-07-23

Этот проект — квази-A/B анализ на симулированных данных о продажах 10 магазинов. Цель — оценить, влияет ли проведение промо-акций (`Promo = 1`) на объём ежедневных продаж.

**Источник данных:** [Kaggle Store Sales Dataset](https://www.kaggle.com/datasets/abhishekjaiswal4896/store-sales-dataset)

---


## 📥 Загрузка и подготовка данных

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set(style='whitegrid')

# Загрузка CSV
file_path = 'store_sales.csv'
df = pd.read_csv(file_path)
df.rename(columns={
    'date': 'Date',
    'store': 'Store_ID',
    'sales': 'Sales',
    'promo': 'Promo',
    'holiday': 'Holiday'
}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.head()

## 📊 Разведочный анализ (все магазины и дни)

In [ ]:
# Сводная статистика
summary_all = df.groupby('Promo')['Sales'].agg(['mean', 'median', 'count']).reset_index()
display(summary_all)

In [ ]:
# Гистограмма
plt.figure(figsize=(10, 5))
sns.histplot(data=df, x='Sales', hue='Promo', bins=50, kde=True, palette='Set1')
plt.title('Распределение продаж по группам Promo')
plt.xlabel('Сумма продаж')
plt.ylabel('Количество дней')
plt.show()

In [ ]:
# Boxplot
plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='Promo', y='Sales', palette='Set2')
plt.title('Boxplot продаж по Promo (все магазины)')
plt.xlabel('Promo')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Time series
plt.figure(figsize=(12, 5))
sns.lineplot(data=df, x='Date', y='Sales', hue='Promo', palette='Set1')
plt.title('Динамика продаж по Promo (все магазины)')
plt.xlabel('Дата')
plt.ylabel('Продажи')
plt.show()

## 🧪 A/B-тест по всем магазинам

In [ ]:
sales_with = df[df['Promo'] == 1]['Sales']
sales_without = df[df['Promo'] == 0]['Sales']

# Welch t-test
t_stat, p_val = stats.ttest_ind(sales_with, sales_without, equal_var=False)
diff = sales_with.mean() - sales_without.mean()

print(f't-статистика: {t_stat:.2f}')
print(f'p-value: {p_val:.2e}')
print(f'Разница средних: {diff:.2f}')

## 📌 Проверка устойчивости: магазин 1, без праздников

In [ ]:
# Фильтрация
store1 = df[(df['Store_ID'] == 1) & (df['Holiday'] == 0)]
store1.groupby('Promo')['Sales'].agg(['mean', 'median', 'count'])

In [ ]:
# Boxplot
plt.figure(figsize=(8, 5))
sns.boxplot(data=store1, x='Promo', y='Sales', palette='Set3')
plt.title('Boxplot: Store 1 без праздников')
plt.xlabel('Promo')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Time series
plt.figure(figsize=(12, 5))
sns.lineplot(data=store1, x='Date', y='Sales', hue='Promo')
plt.title('Динамика продаж: Store 1, обычные дни')
plt.xlabel('Дата')
plt.ylabel('Продажи')
plt.show()

In [ ]:
# Тест для Store 1
s1_with = store1[store1['Promo'] == 1]['Sales']
s1_without = store1[store1['Promo'] == 0]['Sales']
t_s1, p_s1 = stats.ttest_ind(s1_with, s1_without, equal_var=False)
diff_s1 = s1_with.mean() - s1_without.mean()

print(f'Store 1 t-статистика: {t_s1:.2f}')
print(f'p-value: {p_s1:.2e}')
print(f'Разница средних: {diff_s1:.2f}')

## 📚 Методология и выводы


- Это **квази-A/B анализ** — нет рандомизации, мы работаем с наблюдательными данными.
- Использован **Welch t-test**, так как группы неравные по размеру.
- Эффект от промо устойчив как в среднем по всем магазинам, так и при анализе одного магазина без праздников.

---

### 💡 Рекомендации:

- Использовать промо-акции как инструмент краткосрочного стимулирования продаж.
- Уточнить эффект на разных магазинах и в разные сезоны.
- В идеале — провести настоящий A/B-тест с рандомизацией.
